In [28]:
# Map nbdc matchups

In [29]:
import json
import boto3
import sys
!{sys.executable} -m pip install PyAthena
import plotly
from pyathena import connect
import pandas as pd
from pyathena.async_cursor import AsyncCursor

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [30]:
def q2df(q):
    cursor = connect(s3_staging_dir='s3://esdis-ems-athena', region_name='us-west-2', cursor_class=AsyncCursor).cursor(max_workers=2)
    query_id, future = cursor.execute(q)
    print (query_id, future)
#i=0
#for row in cursor:
#    if i==0:
#        print(row)
#    i+=1
#pd.options.display.float_format = '{:,.4f}'.format
#df = pd.read_sql(q, conn)
#df
    result_set = future.result()
    print(result_set.state)
    print(result_set.state_change_reason)
    print(result_set.completion_date_time)
    print(result_set.submission_date_time)
    print('Data Scanned Bytes:', result_set.data_scanned_in_bytes)
    print(result_set.engine_execution_time_in_millis)
    print(result_set.query_queue_time_in_millis)
    print('Total Execution Time (ms):', result_set.total_execution_time_in_millis)
    print(result_set.query_planning_time_in_millis)
    print(result_set.service_processing_time_in_millis)
    print('Output location:', result_set.output_location)
    print(result_set.description)


    s3_client = boto3.client('s3')

    obj=result_set.output_location[result_set.output_location.rfind('/')+1:]
    print(obj)

    s3_client.download_file('esdis-ems-athena', obj, 'myOut.csv')

    df=pd.read_csv('myOut.csv')  
    return df    

In [31]:
# Revised to get nearest and within x distance.
# Stopped after 10 minute execution
# Continue next week.
# Will need to save to table.
q="""
select 
 obstime, 
 st_x(argo_point) buoy_lon,
 st_y(argo_point) buoy_lat,
 temp,
 temp_qc,
 time_base,
 st_x(g16_point) g16_lon,
 st_y(g16_point) g16_lat,
 (sea_surface_temperature - 273.15) sea_surface_temperature_C,
 (sea_surface_temperature - 273.15) - temp temp_diff_C 
from 
(
select 
 obstime, 
 st_point(a.longitude, a.latitude) argo_point, 
 temp, 
 temp_qc,
 time_base,
 st_point(g.longitude, g.latitude) g16_point,
 sea_surface_temperature
from 
 sci.abi_g16_star_l2p_v2_70 g,
 environmental_datasets.argo_nearest_surface a
where 
 a.obstime between timestamp '2021-08-21' and timestamp '2021-08-31'
 and a.longitude between -150 and 0
 and time_base between timestamp '2021-08-21' and timestamp '2021-08-31'
 and g.time_base between a.obstime - interval '30' minute and a.obstime + interval '30' minute
 and st_distance(st_point(g.longitude, g.latitude), st_point(a.longitude, a.latitude)) <= 0.03
) t
order by
 time_base,
 obstime
 """
df=q2df(q)
df

5d626d02-6e1a-47a7-b960-fa75cb41f5ae <Future at 0x7f8f59648e10 state=running>
SUCCEEDED
None
2021-12-06 18:58:34.110000+00:00
2021-12-06 18:57:46.647000+00:00
Data Scanned Bytes: 6530809120
47260
162
Total Execution Time (ms): 47463
1107
41
Output location: s3://esdis-ems-athena/5d626d02-6e1a-47a7-b960-fa75cb41f5ae.csv
[('obstime', 'timestamp', None, None, 3, 0, 'UNKNOWN'), ('buoy_lon', 'double', None, None, 17, 0, 'UNKNOWN'), ('buoy_lat', 'double', None, None, 17, 0, 'UNKNOWN'), ('temp', 'float', None, None, 17, 0, 'UNKNOWN'), ('temp_qc', 'integer', None, None, 10, 0, 'UNKNOWN'), ('time_base', 'timestamp', None, None, 3, 0, 'UNKNOWN'), ('g16_lon', 'double', None, None, 17, 0, 'UNKNOWN'), ('g16_lat', 'double', None, None, 17, 0, 'UNKNOWN'), ('sea_surface_temperature_C', 'double', None, None, 17, 0, 'UNKNOWN'), ('temp_diff_C', 'double', None, None, 17, 0, 'UNKNOWN')]
5d626d02-6e1a-47a7-b960-fa75cb41f5ae.csv


,obstime,buoy_lon,buoy_lat,temp,temp_qc,time_base,g16_lon,g16_lat,sea_surface_temperature_C,temp_diff_C
0,2021-08-21 08:59:05.000,-101.616997,-0.223000,21.634,1,2021-08-21 09:00:00.000,-101.614464,-0.193900,21.119989,-0.514012
1,2021-08-21 08:59:05.000,-101.616997,-0.223000,21.634,1,2021-08-21 09:00:00.000,-101.593063,-0.230820,21.119989,-0.514012
2,2021-08-21 08:59:05.000,-101.616997,-0.223000,21.634,1,2021-08-21 09:00:00.000,-101.593018,-0.212360,21.159998,-0.474003
3,2021-08-21 09:11:55.000,-85.304001,-30.538000,16.431,1,2021-08-21 09:00:00.000,-85.296112,-30.541250,16.260004,-0.170996
4,2021-08-21 09:27:53.000,-109.096001,3.010000,25.998,1,2021-08-21 09:00:00.000,-109.109421,3.018770,25.809991,-0.188008
...,...,...,...,...,...,...,...,...,...,...
1190,2021-08-30 23:42:29.000,-72.396004,24.037001,30.222,1,2021-08-31 00:00:00.000,-72.381104,24.056061,29.740015,-0.481985
1191,2021-08-30 23:51:45.000,-119.537003,-9.823000,25.564,1,2021-08-31 00:00:00.000,-119.561523,-9.812030,25.600000,0.036001
1192,2021-08-30 23:51:45.000,-119.537003,-9.823000,25.564,1,2021-08-31 00:00:00.000,-119.539017,-9.850470,25.580011,0.016012
1193,2021-08-30 23:51:45.000,-119.537003,-9.823000,25.564,1,2021-08-31 00:00:00.000,-119.534729,-9.830930,25.570001,0.006002


In [32]:
df_buoys=df.groupby(["buoy_lon", "buoy_lat", "obstime"], as_index=False).mean()
df_buoys

,buoy_lon,buoy_lat,obstime,temp,temp_qc,g16_lon,g16_lat,sea_surface_temperature_C,temp_diff_C
0,-133.966003,-0.772,2021-08-23 09:09:48.000,23.695,1,-133.952677,-0.777710,24.040002,0.345003
1,-133.776001,0.132,2021-08-27 08:26:43.000,23.671,1,-133.789673,0.127920,23.929997,0.258997
2,-133.292007,1.340,2021-08-28 10:35:24.000,25.270,1,-133.281921,1.346567,25.486668,0.216667
3,-132.449997,12.945,2021-08-25 21:22:37.000,28.091,1,-132.447639,12.951405,27.490015,-0.600985
4,-131.917999,-8.995,2021-08-23 20:48:00.000,26.401,1,-131.915324,-8.991023,26.276666,-0.124333
...,...,...,...,...,...,...,...,...,...
263,-21.681000,-10.055,2021-08-29 08:51:00.000,25.680,1,-21.677340,-10.052560,25.559991,-0.120009
264,-20.389999,-7.645,2021-08-30 04:41:00.000,26.409,1,-20.393656,-7.637927,25.869999,-0.539001
265,-19.336000,-5.431,2021-08-24 17:58:19.000,26.568,1,-19.342617,-5.430687,26.023330,-0.544671
266,-18.542000,-20.503,2021-08-24 14:52:00.000,22.342,1,-18.545920,-20.500057,21.969995,-0.372004


In [33]:
print (min(df['temp_diff_C']),max(df['temp_diff_C']))

-1.297006988525368 1.246999359130882


In [1]:
# conda install -c conda-forge python-kaleido

from IPython.display import display, HTML
import plotly.express as px
import plotly.io as pio
#pio.kaleido.scope.mathjax = None
import plotly.graph_objects as go
plotly.offline.init_notebook_mode(connected=True)
pio.renderers.default='notebook'

fig=go.Figure()
#fig = px.scatter_mapbox(
#                        df, 
#                        lon="irwin_initialLongitude", 
#                        lat="irwin_initialLatitude", 
#                        hover_name="irwin_incidentname", 
#                        hover_data=["irwin_fireDiscoveryDateTime", "irwin_poostate", "filesDistributed"],
#                        color_discrete_sequence=["red"],
#                        size=[ 5 for i in range( df.shape[0] ) ],
#                        zoom=3)


fig = fig.add_trace(go.Scattermapbox(
    lon=round(df['g16_lon'],3), 
    lat=round(df['g16_lat'],3),
    mode='markers',
    marker=go.scattermapbox.Marker(
            size=8,
            color=df['temp_diff_C'],
#            cmax=max(df['sea_surface_temperature']),
#            cmin=min(df['sea_surface_temperature']),
            cmax=2.0,
            cmin=-2.0,
            colorscale='Jet'
    ),
#    marker=go.scattermapbox.Marker(
#            size=10,
#    color_continuous_scale="Viridis",
#    range_color=(min(sst_mean), max(sst_mean)),
#            opacity=0.7
#    ),
    name='SST Aug 21 - 31, 2021',
    text=['diff: ' + str(round(df["temp_diff_C"][i],2)) + '<br>g16_sst: ' + str(round(df["sea_surface_temperature_C"][i],2)) + '<br>nbdc_sst: ' + str(round(df["temp"][i],2)) for i, row in df.iterrows() ],
    hoverinfo='text'
    )
)

fig = fig.add_trace(go.Scattermapbox(
    lon=round(df_buoys['buoy_lon'],2), 
    lat=round(df_buoys['buoy_lat'],2),
    mode='markers',
    marker=go.scattermapbox.Marker(
            size=10,
            color=round(df_buoys['temp_diff_C'], 2),
            cmax=2.0,
            cmin=-2.0,
            colorscale='Jet'
    ),
    name='Buoy',
    text=[ '<br>Water Temperature (C): ' + str(round(df_buoys["temp"][i],2))  for i in range(df_buoys.shape[0]) ],
    hoverinfo='text'    )
)

fig.update_layout(mapbox_style="open-street-map",  height=3000)
fig.update_layout(autosize=False, 
    width=2000, 
    height=1200,
    margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(
    mapbox=dict(bearing=0, center=go.layout.mapbox.Center(lat=20, lon=-75),pitch=0,zoom=5)
)
#fig.show("notebook")
fig.write_html("map_" + "sst_g16_matchup" + ".html")
#fig.update_layout(zoom=10)
#fig.write_image("map_" + "sst_g16" + ".jpg", scale=3)
#pio.write_image(fig, file='map_sst_g16.jpg', format="jpg")

NameError: name 'plotly' is not defined

In [35]:
# Matchup Reduce SQL Example
q="""
select
  obstime, 
  buoy_lon,
  buoy_lat,
 temp,
 temp_qc,
 t3.reduced.time_base,
 t3.reduced.g16_lon,
 t3.reduced.g16_lat,
 t3.reduced.sea_surface_temperature_c,
 t3.reduced.obs_spatial_distance 
from (
 select 
 t2.*,
 reduce(
        matches,
        CAST(
            Row(
   now(), 
   9999.9, 
   9999.9, 
   -9999.0, 
   9999.9
            ) AS 
            ROW(
   time_base timestamp, 
   g16_lon real, 
   g16_lat real, 
   sea_surface_temperature_c real, 
   obs_spatial_distance real
            )
        ),
        (state, matches) -> IF(matches.obs_spatial_distance < state.obs_spatial_distance, matches, state),
        state -> state
) reduced
--SELECT reduce(ARRAY [5, 20, NULL, 50], 0, (s, x) -> IF(x IS NULL, s, s + x), s -> s); -- 75
from 
(
select 
 obstime, 
 st_x(argo_point) buoy_lon,
 st_y(argo_point) buoy_lat,
 temp,
 temp_qc,
 array_agg(
 CAST(
  Row(
   time_base, 
   st_x(g16_point), 
   st_y(g16_point), 
   sea_surface_temperature - 273.15, 
   st_distance(g16_point, argo_point )
   ) AS 
  ROW(
   time_base timestamp, 
   g16_lon real, 
   g16_lat real, 
   sea_surface_temperature_c real, 
   obs_spatial_distance real
   )
  )
 ) matches
from 
(
select 
 obstime, 
 st_point(a.longitude, a.latitude) argo_point, 
 temp, 
 temp_qc,
 time_base,
 st_point(g.longitude, g.latitude) g16_point,
 sea_surface_temperature
from 
 sci.abi_g16_star_l2p_v2_70 g,
 environmental_datasets.argo_nearest_surface a
where 
 a.obstime between timestamp '2021-08-21' and timestamp '2021-08-31'
 and a.longitude between -150 and 0
 and time_base between timestamp '2021-08-21' and timestamp '2021-08-31'
 and g.time_base between a.obstime - interval '30' minute and a.obstime + interval '30' minute
 and st_distance(st_point(g.longitude, g.latitude), st_point(a.longitude, a.latitude)) <= 0.03
) t
group by
 obstime, 
 st_x(argo_point),
 st_y(argo_point),
 temp,
 temp_qc
order by
 obstime,
 st_x(argo_point)
 ) t2
) t3
"""
df=q2df(q)
df

ba9c9e5d-0d4b-490a-960c-1c1392a47f5b <Future at 0x7f8f5ab8ad30 state=running>
SUCCEEDED
None
2021-12-06 19:00:11.107000+00:00
2021-12-06 18:59:22.416000+00:00
Data Scanned Bytes: 6530809120
48420
204
Total Execution Time (ms): 48691
890
67
Output location: s3://esdis-ems-athena/ba9c9e5d-0d4b-490a-960c-1c1392a47f5b.csv
[('obstime', 'timestamp', None, None, 3, 0, 'UNKNOWN'), ('buoy_lon', 'double', None, None, 17, 0, 'UNKNOWN'), ('buoy_lat', 'double', None, None, 17, 0, 'UNKNOWN'), ('temp', 'float', None, None, 17, 0, 'UNKNOWN'), ('temp_qc', 'integer', None, None, 10, 0, 'UNKNOWN'), ('time_base', 'timestamp', None, None, 3, 0, 'UNKNOWN'), ('g16_lon', 'float', None, None, 17, 0, 'UNKNOWN'), ('g16_lat', 'float', None, None, 17, 0, 'UNKNOWN'), ('sea_surface_temperature_c', 'float', None, None, 17, 0, 'UNKNOWN'), ('obs_spatial_distance', 'float', None, None, 17, 0, 'UNKNOWN')]
ba9c9e5d-0d4b-490a-960c-1c1392a47f5b.csv


,obstime,buoy_lon,buoy_lat,temp,temp_qc,time_base,g16_lon,g16_lat,sea_surface_temperature_c,obs_spatial_distance
0,2021-08-21 08:59:05.000,-101.616997,-0.223000,21.634,1,2021-08-21 09:00:00.000,-101.59306,-0.23082,21.119990,0.025179
1,2021-08-21 09:11:55.000,-85.304001,-30.538000,16.431,1,2021-08-21 09:00:00.000,-85.29611,-30.54125,16.260004,0.008532
2,2021-08-21 09:27:53.000,-109.096001,3.010000,25.998,1,2021-08-21 09:00:00.000,-109.08517,3.01864,25.869990,0.013857
3,2021-08-21 10:19:21.000,-38.777000,-0.933000,27.279,1,2021-08-21 10:00:00.000,-38.77926,-0.92809,27.230005,0.005404
4,2021-08-21 12:06:00.000,-66.307999,12.362000,28.996,1,2021-08-21 12:00:00.000,-66.30999,12.36222,29.010004,0.002003
...,...,...,...,...,...,...,...,...,...,...
263,2021-08-30 21:09:36.000,-96.847000,22.504999,30.122,4,2021-08-30 21:00:00.000,-96.85403,22.49409,29.890009,0.012977
264,2021-08-30 22:29:45.000,-53.171001,7.916000,29.428,1,2021-08-30 22:00:00.000,-53.17501,7.92202,29.860010,0.007233
265,2021-08-30 22:32:45.000,-88.492996,-2.508000,20.286,1,2021-08-30 23:00:00.000,-88.48894,-2.48727,20.249994,0.021123
266,2021-08-30 23:42:29.000,-72.396004,24.037001,30.222,1,2021-08-31 00:00:00.000,-72.40119,24.05599,29.790003,0.019685


In [45]:
q="""
with t1 as ( 
 select 
 obstime, 
 st_point(a.longitude, a.latitude) argo_point, 
 temp, 
 temp_qc,
 time_base,
 st_point(g.longitude, g.latitude) g16_point,
 sea_surface_temperature,
 min( st_distance(st_point(g.longitude, g.latitude), st_point(a.longitude, a.latitude)) ) 
 OVER (
  PARTITION BY obstime, a.longitude, a.latitude, temp 
  ORDER BY obstime, a.longitude, a.latitude, temp 
 ) as nearest_obs,
 st_distance(st_point(g.longitude, g.latitude), st_point(a.longitude, a.latitude)) obs_distance
from 
 sci.abi_g16_star_l2p_v2_70 g,
 environmental_datasets.argo_nearest_surface a
where 
 a.obstime between timestamp '2021-08-21' and timestamp '2021-08-31'
 and a.longitude between -150 and 0
 and time_base between timestamp '2021-08-21' and timestamp '2021-08-31'
 and g.time_base between a.obstime - interval '30' minute and a.obstime + interval '30' minute
 and st_distance(st_point(g.longitude, g.latitude), st_point(a.longitude, a.latitude)) <= 0.03
 ) 
select 
 obstime, 
 st_x(argo_point) buoy_lon,
 st_y(argo_point) buoy_lat,
 temp,
 time_base, 
 st_x(g16_point) g16_lon, 
 st_y(g16_point) g16_lat, 
 sea_surface_temperature - 273.15 sea_surface_temperature_C, 
 obs_distance,
 nearest_obs,
 (sea_surface_temperature - 273.15) - temp temp_diff_C 
from 
 t1
where
 obs_distance = nearest_obs
order by
 obstime,
 st_x(argo_point)
"""
df=q2df(q)
df

058adcdd-4291-4650-af82-e9812e72b9ec <Future at 0x7f8f5cfc0160 state=running>
SUCCEEDED
None
2021-12-06 19:09:27.147000+00:00
2021-12-06 19:08:40.243000+00:00
Data Scanned Bytes: 6530528411
46693
160
Total Execution Time (ms): 46904
905
51
Output location: s3://esdis-ems-athena/058adcdd-4291-4650-af82-e9812e72b9ec.csv
[('obstime', 'timestamp', None, None, 3, 0, 'UNKNOWN'), ('buoy_lon', 'double', None, None, 17, 0, 'UNKNOWN'), ('buoy_lat', 'double', None, None, 17, 0, 'UNKNOWN'), ('temp', 'float', None, None, 17, 0, 'UNKNOWN'), ('time_base', 'timestamp', None, None, 3, 0, 'UNKNOWN'), ('g16_lon', 'double', None, None, 17, 0, 'UNKNOWN'), ('g16_lat', 'double', None, None, 17, 0, 'UNKNOWN'), ('sea_surface_temperature_C', 'double', None, None, 17, 0, 'UNKNOWN'), ('obs_distance', 'double', None, None, 17, 0, 'UNKNOWN'), ('nearest_obs', 'double', None, None, 17, 0, 'UNKNOWN'), ('temp_diff_C', 'double', None, None, 17, 0, 'UNKNOWN')]
058adcdd-4291-4650-af82-e9812e72b9ec.csv


,obstime,buoy_lon,buoy_lat,temp,time_base,g16_lon,g16_lat,sea_surface_temperature_C,obs_distance,nearest_obs,temp_diff_C
0,2021-08-21 08:59:05.000,-101.616997,-0.223000,21.634,2021-08-21 09:00:00.000,-101.593063,-0.230820,21.119989,0.025179,0.025179,-0.514012
1,2021-08-21 09:11:55.000,-85.304001,-30.538000,16.431,2021-08-21 09:00:00.000,-85.296112,-30.541250,16.260004,0.008532,0.008532,-0.170996
2,2021-08-21 09:27:53.000,-109.096001,3.010000,25.998,2021-08-21 09:00:00.000,-109.085167,3.018640,25.869989,0.013857,0.013857,-0.128010
3,2021-08-21 10:19:21.000,-38.777000,-0.933000,27.279,2021-08-21 10:00:00.000,-38.779259,-0.928090,27.230005,0.005404,0.005404,-0.048994
4,2021-08-21 12:06:00.000,-66.307999,12.362000,28.996,2021-08-21 12:00:00.000,-66.309990,12.362220,29.010004,0.002003,0.002003,0.014003
...,...,...,...,...,...,...,...,...,...,...,...
263,2021-08-30 21:09:36.000,-96.847000,22.504999,30.122,2021-08-30 21:00:00.000,-96.854027,22.494089,29.890009,0.012977,0.012977,-0.231991
264,2021-08-30 22:29:45.000,-53.171001,7.916000,29.428,2021-08-30 22:00:00.000,-53.175011,7.922020,29.860010,0.007233,0.007233,0.432010
265,2021-08-30 22:32:45.000,-88.492996,-2.508000,20.286,2021-08-30 23:00:00.000,-88.488937,-2.487270,20.249994,0.021123,0.021123,-0.036005
266,2021-08-30 23:42:29.000,-72.396004,24.037001,30.222,2021-08-31 00:00:00.000,-72.401192,24.055990,29.790002,0.019685,0.019685,-0.431998


In [46]:
df_buoys=df.groupby(["buoy_lon", "buoy_lat", "obstime"], as_index=False).mean()
df_buoys

,buoy_lon,buoy_lat,obstime,temp,g16_lon,g16_lat,sea_surface_temperature_C,obs_distance,nearest_obs,temp_diff_C
0,-133.966003,-0.772,2021-08-23 09:09:48.000,23.695,-133.952667,-0.777710,24.059991,0.014507,0.014507,0.364992
1,-133.776001,0.132,2021-08-27 08:26:43.000,23.671,-133.789673,0.127920,23.920007,0.014268,0.014268,0.249008
2,-133.292007,1.340,2021-08-28 10:35:24.000,25.270,-133.281921,1.346570,25.480005,0.012037,0.012037,0.210004
3,-132.449997,12.945,2021-08-25 21:22:37.000,28.091,-132.442444,12.941100,27.490015,0.008500,0.008500,-0.600985
4,-131.917999,-8.995,2021-08-23 20:48:00.000,26.401,-131.915314,-8.991020,26.189996,0.004801,0.004801,-0.211003
...,...,...,...,...,...,...,...,...,...,...
263,-21.681000,-10.055,2021-08-29 08:51:00.000,25.680,-21.674120,-10.062550,25.559991,0.010214,0.010214,-0.120009
264,-20.389999,-7.645,2021-08-30 04:41:00.000,26.409,-20.393660,-7.637930,25.860010,0.007961,0.007961,-0.548991
265,-19.336000,-5.431,2021-08-24 17:58:19.000,26.568,-19.342621,-5.430690,26.029993,0.006628,0.006628,-0.538008
266,-18.542000,-20.503,2021-08-24 14:52:00.000,22.342,-18.551060,-20.514009,21.980005,0.014258,0.014258,-0.361994


In [47]:
from IPython.display import display, HTML
import plotly.express as px
import plotly.io as pio
#pio.kaleido.scope.mathjax = None
import plotly.graph_objects as go
plotly.offline.init_notebook_mode(connected=True)
pio.renderers.default='notebook'

fig=go.Figure()
#fig = px.scatter_mapbox(
#                        df, 
#                        lon="irwin_initialLongitude", 
#                        lat="irwin_initialLatitude", 
#                        hover_name="irwin_incidentname", 
#                        hover_data=["irwin_fireDiscoveryDateTime", "irwin_poostate", "filesDistributed"],
#                        color_discrete_sequence=["red"],
#                        size=[ 5 for i in range( df.shape[0] ) ],
#                        zoom=3)


fig = fig.add_trace(go.Scattermapbox(
    lon=round(df['g16_lon'],3), 
    lat=round(df['g16_lat'],3),
    mode='markers',
    marker=go.scattermapbox.Marker(
            size=8,
            color=df['temp_diff_C'],
#            cmax=max(df['sea_surface_temperature']),
#            cmin=min(df['sea_surface_temperature']),
            cmax=2.0,
            cmin=-2.0,
            colorscale='Jet'
    ),
#    marker=go.scattermapbox.Marker(
#            size=10,
#    color_continuous_scale="Viridis",
#    range_color=(min(sst_mean), max(sst_mean)),
#            opacity=0.7
#    ),
    name='SST Aug 21 - 31, 2021',
    text=['diff: ' + str(round(df["temp_diff_C"][i],2)) + '<br>g16_sst: ' + str(round(df["sea_surface_temperature_C"][i],2)) + '<br>nbdc_sst: ' + str(round(df["temp"][i],2)) for i, row in df.iterrows() ],
    hoverinfo='text'
    )
)

fig = fig.add_trace(go.Scattermapbox(
    lon=round(df_buoys['buoy_lon'],2), 
    lat=round(df_buoys['buoy_lat'],2),
    mode='markers',
    marker=go.scattermapbox.Marker(
            size=10,
            color=round(df_buoys['temp_diff_C'], 2),
            cmax=2.0,
            cmin=-2.0,
            colorscale='Jet'
    ),
    name='Buoy',
    text=[ '<br>Water Temperature (C): ' + str(round(df_buoys["temp"][i],2))  for i in range(df_buoys.shape[0]) ],
    hoverinfo='text'    )
)

fig.update_layout(mapbox_style="open-street-map",  height=3000)
fig.update_layout(autosize=False, 
    width=2000, 
    height=1200,
    margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(
    mapbox=dict(bearing=0, center=go.layout.mapbox.Center(lat=20, lon=-75),pitch=0,zoom=5)
)
fig.write_html("map_" + "sst_g16_matchup_nearest" + ".html")


In [49]:
q="""
with t1 as ( 
 select
  station,
  longitude,
  latitude,
  obstime,
  wtmp,
  pres
from environmental_datasets.ocean_bouy_nbdc
where 
 obstime between timestamp '2021-08-22' and timestamp '2021-09-01'
 and longitude between -95.0 and 60.0
)
select station, longitude, latitude, min(obstime), max(obstime), count(*) 
from t1 
group by station, longitude, latitude 
order by 6 desc
"""
df=q2df(q)
df

f7a7ca81-949c-48b9-b35f-de7a87ab5f1b <Future at 0x7f8f5b9a65f8 state=running>
SUCCEEDED
None
2021-12-06 19:47:43.218000+00:00
2021-12-06 19:47:41.015000+00:00
Data Scanned Bytes: 451315308
1793
209
Total Execution Time (ms): 2203
None
201
Output location: s3://esdis-ems-athena/f7a7ca81-949c-48b9-b35f-de7a87ab5f1b.csv
[('station', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('longitude', 'float', None, None, 17, 0, 'UNKNOWN'), ('latitude', 'float', None, None, 17, 0, 'UNKNOWN'), ('_col3', 'timestamp', None, None, 3, 0, 'UNKNOWN'), ('_col4', 'timestamp', None, None, 3, 0, 'UNKNOWN'), ('_col5', 'bigint', None, None, 19, 0, 'UNKNOWN')]
f7a7ca81-949c-48b9-b35f-de7a87ab5f1b.csv


,station,longitude,latitude,_col3,_col4,_col5
0,45023,-88.607,47.270,2021-08-22 00:00:00.000,2021-08-31 23:55:00.000,2880
1,45025,-88.398,46.969,2021-08-22 00:00:00.000,2021-08-31 23:55:00.000,2879
2,carl1,-90.135,29.933,2021-08-22 00:00:00.000,2021-08-31 23:54:00.000,2400
3,pnlm6,-88.563,30.368,2021-08-22 00:00:00.000,2021-08-31 23:54:00.000,2400
4,bdrn4,-74.870,40.082,2021-08-22 00:00:00.000,2021-08-31 23:54:00.000,2400
...,...,...,...,...,...,...
293,cmln3,-70.700,43.070,2021-08-22 10:04:00.000,2021-08-31 23:04:00.000,55
294,44078,-39.520,59.940,2021-08-31 15:00:00.000,2021-08-31 23:50:00.000,54
295,44073,-70.540,43.020,2021-08-22 00:04:00.000,2021-08-30 14:04:00.000,33
296,44039,-72.655,41.138,2021-08-31 02:00:00.000,2021-08-31 14:00:00.000,2


In [54]:
from IPython.display import display, HTML
import plotly.express as px
import plotly.io as pio
#pio.kaleido.scope.mathjax = None
import plotly.graph_objects as go
plotly.offline.init_notebook_mode(connected=True)
pio.renderers.default='notebook'

fig=go.Figure()
#fig = px.scatter_mapbox(
#                        df, 
#                        lon="irwin_initialLongitude", 
#                        lat="irwin_initialLatitude", 
#                        hover_name="irwin_incidentname", 
#                        hover_data=["irwin_fireDiscoveryDateTime", "irwin_poostate", "filesDistributed"],
#                        color_discrete_sequence=["red"],
#                        size=[ 5 for i in range( df.shape[0] ) ],
#                        zoom=3)


fig = fig.add_trace(go.Scattermapbox(
    lon=round(df['longitude'],3), 
    lat=round(df['latitude'],3),
    mode='markers',
    marker=go.scattermapbox.Marker(
            size=8,
            color=df['_col5'],
            cmax=max(df['_col5']),
            cmin=min(df['_col5']),
#            cmax=2.0,
#            cmin=-2.0,
            colorscale='Jet'
    ),
#    marker=go.scattermapbox.Marker(
#            size=10,
#    color_continuous_scale="Viridis",
#    range_color=(min(sst_mean), max(sst_mean)),
#            opacity=0.7
#    ),
    name='SST Aug 21 - 31, 2021',
    text=['station: ' + df["station"][i] + '<br>count: ' + str(df["_col5"][i]) for i, row in df.iterrows() ],
    hoverinfo='text'
    )
)

fig.update_layout(mapbox_style="open-street-map",  height=3000)
fig.update_layout(autosize=False, 
    width=2000, 
    height=1200,
    margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(
    mapbox=dict(bearing=0, center=go.layout.mapbox.Center(lat=20, lon=-75),pitch=0,zoom=5)
)
fig.write_html("map_" + "sst_g16_matchup_nearest" + ".html")

In [100]:
q="""
with t1 as ( 
 select 
 station,
 obstime, 
 st_point(a.longitude, a.latitude) argo_point, 
 wtmp, 
 time_base,
 st_point(g.longitude, g.latitude) g16_point,
 sea_surface_temperature
from 
 sci.abi_g16_star_l2p_v2_70 g,
 environmental_datasets.ocean_bouy_nbdc a
where 
 station in ('42058', '42056', '42003', '42099')
 and a.obstime between timestamp '2021-08-21' and timestamp '2021-08-31'
 and a.longitude between -150 and 0
 and time_base between timestamp '2021-08-22' and timestamp '2021-08-31'
 and g.time_base between a.obstime - interval '30' minute and a.obstime + interval '30' minute
 and st_distance(st_point(g.longitude, g.latitude), st_point(a.longitude, a.latitude)) <= 0.03
 ) 
select 
 station,
 obstime,
 avg(wtmp),
 avg(sea_surface_temperature),
 avg((sea_surface_temperature - 273.15) - wtmp )
from 
 t1
group by
 station,obstime
order by
 station,obstime
"""
df=q2df(q)

ba813e3a-4a7c-4daf-96ef-1664c104a0d6 <Future at 0x7f8f59ae6f28 state=running>
SUCCEEDED
None
2021-12-06 20:56:32.061000+00:00
2021-12-06 20:55:52.403000+00:00
Data Scanned Bytes: 6452118621
39483
131
Total Execution Time (ms): 39658
931
44
Output location: s3://esdis-ems-athena/ba813e3a-4a7c-4daf-96ef-1664c104a0d6.csv
[('station', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('obstime', 'timestamp', None, None, 3, 0, 'UNKNOWN'), ('_col2', 'float', None, None, 17, 0, 'UNKNOWN'), ('_col3', 'float', None, None, 17, 0, 'UNKNOWN'), ('_col4', 'double', None, None, 17, 0, 'UNKNOWN')]
ba813e3a-4a7c-4daf-96ef-1664c104a0d6.csv


In [113]:
pvtable = pd.pivot_table(df, values='_col4', index=['obstime'],
                    columns=['station'], aggfunc=sum)

In [115]:
for i,r in pvtable.iterrows():
    print ('i=',i,'\nr=',r)
    print (i)
    print (r[42003])
#    print (r[42058])
#    print (r[42056])
#    print (r[42099])
    break
print (pvtable.columns)
print (df.columns)

i= 2021-08-21 23:30:00.000 
r= station
42003        NaN
42056        NaN
42058   -0.10572
42099        NaN
Name: 2021-08-21 23:30:00.000, dtype: float64
2021-08-21 23:30:00.000
nan
Int64Index([42003, 42056, 42058, 42099], dtype='int64', name='station')
Index(['station', 'obstime', '_col2', '_col3', '_col4'], dtype='object')


In [116]:
pvtable['index1']=pvtable.index
import plotly.express as px

fig = px.line(pvtable, x='index1', y=pvtable.columns)
#fig.show()
fig.write_html("carrib_bouys_time_series" + ".html")